In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.optim import Adam
from sklearn.preprocessing import StandardScaler


np.random.seed(42) # 乱数生成用のシードを設定
ite = 1 # 乱数生成の回数

random_seed = np.random.randint(0, 10000, ite)  # ランダムな整数値をシード値として取得.例えば 0 〜 9999 の間の整数をite個生成
print("random_seed", random_seed) # 乱数シード値の確認


In [ ]:
##################################################################

# 時間埋め込み（正弦波位置エンコーディング）
def pos_encoding(timesteps, output_dim, device='cpu'):
    position = timesteps.view(-1, 1).float()  # 必要に応じて型変換
    div_term = torch.exp(torch.arange(0, output_dim, 2, device=device, dtype=torch.float32) * 
                         (-np.log(10000.0) / output_dim))
    sinusoid = torch.cat([torch.sin(position * div_term), torch.cos(position * div_term)], dim=1)
    return sinusoid

# Dropoutの導入: 過学習を防ぐために、各隠れ層にnn.Dropoutを追加。
# Batch Normalizationの導入: 学習を安定させるためにnn.BatchNorm1dを適用。
# 活性化関数の選択: F.reluの代わりにnn.LeakyReLUやnn.ELUを試すことで、勾配消失問題に対応。

# 拡散モデル
class DiffusionModel(nn.Module):
    def __init__(self, time_embed_dim=16):
        super(DiffusionModel, self).__init__()
        self.time_embed_dim = time_embed_dim  # time_embed_dimをインスタンス変数として初期化
        self.fc1 = nn.Linear(2 + time_embed_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x, t):
        # 時間埋め込み
        t_embed = pos_encoding(t, self.time_embed_dim, x.device)
        x_t = torch.cat([x, t_embed], dim=-1)  # 時間情報と入力データを結合
        x_t = F.relu(self.fc1(x_t))
        x_t = F.relu(self.fc2(x_t))
        return self.fc3(x_t)

# 拡散プロセス
class Diffuser:
    def __init__(self, num_timesteps=1000, beta_start=0.0001, beta_end=0.02, device='cpu'):
        self.num_timesteps = num_timesteps
        self.device = device
        self.betas = torch.linspace(beta_start, beta_end, num_timesteps, device=device)
        self.alphas = 1 - self.betas
        self.alpha_bars = torch.cumprod(self.alphas, dim=0)

    def add_noise(self, x_0, t):
        t_idx = t - 1 # alphas[0] is for t=1
        alpha_bar = self.alpha_bars[t_idx].view(-1, 1)  # (N, 1)
        noise = torch.randn_like(x_0, device=self.device)
        x_t = torch.sqrt(alpha_bar) * x_0 + torch.sqrt(1 - alpha_bar) * noise
        return x_t, noise

    def denoise(self, model, x, t):
        T = self.num_timesteps
        assert (t >= 1).all() and (t <= T).all()
        
        t_idx = t - 1 # alphas[0] is for t=1
        alpha = self.alphas[t_idx].view(-1, 1)
        alpha_bar = self.alpha_bars[t_idx].view(-1, 1)
        model.eval()
        with torch.no_grad():
            eps = model(x, t)

        noise = torch.randn_like(x, device=self.device)
        noise[t == 1] = 0  # no noise at t=1

        mu = (x - (1 - alpha) / torch.sqrt(1 - alpha_bar) * eps) / torch.sqrt(alpha)

        return mu

In [ ]:
# 例4: 比率30:70、円形クラスタ
print("--- Example 4: 30:70 Ratio, Circular Clusters ---")
dataset4 = generate_uncorrelated_gmm_2d_toy_dataset(
    num_total_samples=2000,
    ratio_mode1=0.3,
    mu1=np.array([-3.0, -3.0]),
    sigma1_diag=np.array([1.0, 1.0]),
    mu2=np.array([3.0, 3.0]),
    sigma2_diag=np.array([1.0, 1.0])
)
plot_2d_dataset(dataset4, title="Uncorrelated GMM (30:70, Circular)")

In [ ]:
###################################################################

# 2次元正規分布の平均ベクトルと共分散行列を設定
original_mean = [5, 5]  # 平均ベクトル
original_cov = [[1, 0.3], [0.3, 1]]  # 共分散行列（相関あり）

# データセットサイズとデータセット数
num_dataset = 1000 # サンプリング回数
dataset_size = 500 # 元データのサイズ 


# ハイパーパラメータ
num_timesteps = 500 # 拡散ステップ数
epochs = 10          # 学習エポック数
lr = 1e-3         # 学習率
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# シード値の固定
np.random.seed(42)

# サンプリング
iter = 1 #学習元データの数
# シード値の生成
random_seed = np.random.randint(0, 10000, iter)

# モデルとデータを管理する辞書
models = {}
original_datas = {}

for (i, seed) in enumerate(random_seed):
    start_time = time.time() # 計測開始

    # モデルの初期化
    time_embed_dim = 16
    model = DiffusionModel(time_embed_dim=time_embed_dim).to(device)
    optimizer = Adam(model.parameters(), lr=lr)
    diffuser = Diffuser(num_timesteps=num_timesteps, device=device)

    # 学習データ(ガウスノイズ)
    print("############################################")
    print(f"Data_Set_{i+1}, Seed: {seed}") # 開始の合図

    print(f"拡散ステップ数: {num_timesteps}, 学習エポック数: {epochs}, 学習率: {lr}")
    print("############################################")
    np.random.seed(seed) # 取得した乱数を新しいシード値として設定
    data = np.random.multivariate_normal(original_mean, original_cov, size=dataset_size) # 学習元データの生成 (50, 2)
    # データの標準化
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data) # shape: (500, 2), dtype: float64

    print("data.shape", data.shape) # (50, 2)
    # PyTorchテンソルへ明示的に float32 で変換
    train_data = torch.tensor(scaled_data, dtype=torch.float32).to(device) # shape: (50, 2)

    # データローダー作成
    batch_size = 10
    # モデル学習に使う DataLoader も float32 のテンソルから作成
    dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)


    # 原点を基準として赤十字
    plt.axhline(original_mean[0], color='gray', linestyle='--')
    plt.axvline(original_mean[1], color='gray', linestyle='--')
    plt.scatter(original_mean[0], original_mean[1], color='red', marker='x', s=100, label='True Mean')
    # データの可視化
    plt.scatter(data[:, 0], data[:, 1], label='Original Data')
    plt.title(f'Original Data (Seed: {seed})')
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.legend()
    plt.xlim(0, 10)
    plt.ylim(0, 10)
    plt.grid()
    plt.show()

    # 標準化したデータの可視化
    plt.scatter(scaled_data[:, 0], scaled_data[:, 1], label='Scaled Data')
    plt.title(f'Scaled Data (Seed: {seed})')
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.legend()
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.grid()
    plt.show()

    # データの要約
    print("dataの平均ベクトル", np.mean(data, axis=0)) # 平均ベクトル
    print("dataの分散共分散行列", np.cov(data.T)) # 分散共分散行列
    print("dataの相関係数", np.corrcoef(data.T)) # 相関係数

    # 学習
    losses = []
    for epoch in range(epochs):
        loss_sum = 0.0
        for batch in dataloader:
            optimizer.zero_grad()
            x = batch.to(device)
            t = torch.randint(1, num_timesteps + 1, (len(x),), device=device)

            x_noisy, noise = diffuser.add_noise(x, t)
            noise_pred = model(x_noisy, t)
            loss = F.mse_loss(noise_pred, noise)

            loss.backward()
            optimizer.step()

            loss_sum += loss.item()
        avg_loss = loss_sum / len(dataloader)
        losses.append(avg_loss)
        # 5の倍数エポックで損失を表示
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}, Loss: {avg_loss}")
    # 辞書に保存
    models[f"model_{i+1}"] = model
    original_datas[f"seed_{seed}"] = data
    print("学習終了")
    end_time = time.time() # 計測終了
    print('\n')
    print(f"学習時間: {end_time - start_time:.2f}秒")

    # # モデルの保存
    # torch.save(model.state_dict(), f"model_{i+1}.pth")

    # 学習曲線のプロット
    plt.plot(losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Trained By data_by_seed_{}'.format(seed))
    plt.show()
    print('\n')
    print("#"*50)
    print('\n')

In [ ]:
print(models)

In [ ]:
###################################################################


# 拡散モデルのサンプリング関数
from tqdm import tqdm


def generate_samples(model, n=dataset_size, B=num_dataset, device='cpu'):
    model.eval() # モデルを評価モードに設定
    with torch.no_grad(): # 勾配計算を無効化
        new_sample_list = [] # サンプルを格納するリスト
        for _ in tqdm(range(B)): # 進捗バーを表示
            torch.manual_seed(np.random.randint(0, 10000)) # ランダムシードを設定
            samples = torch.randn((n, 2), device=device) # サンプルを生成
            for t in range(num_timesteps, 0, -1): # 拡散ステップを逆に進める
                t_tensor = torch.full((n,), t, device=device, dtype=torch.long) # 時間情報をテンソルに変換
                samples = diffuser.denoise(model, samples, t_tensor) # ノイズ除去
            new_sample_list.append(samples.cpu().numpy()) # CPUに戻してnumpy配列に変換
        return np.array(new_sample_list)  # (B, n, 2) 


# ブートストラップサンプリングの実行
generated_data_list = []


# 拡散モデルによるサンプリング
for model_key, data, seed, selected_model in zip(original_datas.keys(), original_datas.values(), models.keys(), models.values()):
    print("-"*50)
    print("#"*50)
    print(f"Seed: {seed.split('_')[-1]}")
    print("サンプリング開始")

    start_time = time.time() # 計測開始
    generated_data = generate_samples(selected_model, n=dataset_size, B=num_dataset, device=device) # サンプリング実行
    # scalerを使ったならば、逆変換を行う
    if scaler:
        generated_data = scaler.inverse_transform(generated_data.reshape(-1, 2)).reshape(num_dataset, dataset_size, 2)
    generated_data_list.append(generated_data) # サンプルをリストに追加
    generated_data = np.array(generated_data) # numpy配列に変換
    print("generated_data.shape", generated_data.shape) # (B, n, 2)
    end_time = time.time() # 計測終了
    print("サンプリング終了")

    print(f"サンプリング時間: {end_time - start_time:.2f}秒")
    print(f"サンプリング時間: {(end_time - start_time)//60}分 {(end_time - start_time)%60}秒")

    # サンプルされたデータの保存
    # torch.save(generated_data, f"master_research/saved_data/sampled_data/sampled_data_{seed.split('_')[-1]}_epoch_{epochs}.pth")
    print("#"*50)


In [ ]:
print("generated_data_list[0].shape", generated_data_list[0].shape) # (1000, 50, 2)

### 逆変換

In [ ]:
# generated_data: shape (B, n, 2)
# scaler: 学習時に fit() 済の StandardScaler オブジェクト

generated_data = generated_data_list[0]

# 逆変換を適用
generated_data_original = np.array([
    scaler.inverse_transform(sample) for sample in generated_data
])  # shape: (B, n, 2)

generated_data_list[0] = generated_data_original

### データセットごとに図示

In [ ]:
print("generated_data_list[0].shape", generated_data_list[0].shape)  # (1000, 50, 2)
# サンプリングされたデータの可視化
plt.figure(figsize=(10, 8))
plt.scatter(generated_data_list[0][:, 1, 0].flatten(), generated_data_list[0][:, 1, 1].flatten(), alpha=0.5, label='Generated Samples')
plt.title('Generated Samples from Diffusion Model')
plt.xlabel('X1')
plt.ylabel('X2')
plt.axhline(original_mean[0], color='gray', linestyle='--')
plt.axvline(original_mean[1], color='gray', linestyle='--')
plt.scatter(original_mean[0], original_mean[1], color='red', marker='x', s=100, label='True Mean')
plt.xlim(9.0, 11)
plt.ylim(9.0, 11)
plt.legend()
plt.grid()
plt.show()

In [ ]:
sample_sampled_data = generate_samples(models['model_1'], n=dataset_size, B=50, device=device)
# 逆変換を適用
sample_sampled_data = scaler.inverse_transform(sample_sampled_data.reshape(-1, 2)).reshape(1, dataset_size, 2)  # shape: (1, 500, 2)
x_mean_sample_sampled_data = np.mean(sample_sampled_data[0], axis=0)  # サンプルされたデータのx平均
y_mean_sample_sampled_data = np.mean(sample_sampled_data[0], axis=1)  # サンプルされたデータのy平均
# サンプルされたデータの可視化
plt.figure(figsize=(10, 8))
plt.scatter(sample_sampled_data[0][:, 0], sample_sampled_data[0][:, 1], alpha=0.5, label='Sampled Data')
plt.title('Sampled Data from Diffusion Model')
plt.xlabel('X1')
plt.ylabel('X2')
plt.axhline(original_mean[0], color='gray', linestyle='--')
plt.axvline(original_mean[1], color='gray', linestyle='--')
plt.scatter(original_mean[0], original_mean[1], color='red', marker='x', s=100, label='True Mean')
# plt.xlim(x_mean_sample_sampled_data-1, x_mean_sample_sampled_data+1)
# plt.ylim(y_mean_sample_sampled_data-1, y_mean_sample_sampled_data+1)
plt.legend()
plt.grid()
plt.show()

### データセットごとに平均値をとる

In [ ]:
# generated_data_original: shape (1000, 50, 2)
# 生成されたデータのデータセットごとの平均ベクトルと分散共分散行列を計算
means_of_generated_data = np.mean(generated_data_original, axis=1)  # shape: (1000, 2)
# print("生成されたデータのデータセットごとの平均ベクトル(means_of_generated_data)", means_of_generated_data)
print("生成されたデータのデータセットごとの平均ベクトル(means_of_generated_data)のshape", means_of_generated_data.shape)


In [ ]:
## 可視化
x_vals = means_of_generated_data[:, 0].flatten()
y_vals = means_of_generated_data[:, 1].flatten()

# 最小・最大 + 標準偏差から範囲を決定
x_min, x_max = x_vals.min(), x_vals.max()
y_min, y_max = y_vals.min(), y_vals.max()
x_std, y_std = x_vals.std(), y_vals.std()

k = 0.5  # 余白のスケール（標準偏差単位）

# プロット
plt.figure(figsize=(6, 6))
plt.scatter(x_vals, y_vals, alpha=0.5, label='Generated Data')
plt.title('Generated Data (Seed: {})'.format(random_seed[0]))
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()
plt.grid()
plt.axis('equal')

# 拡張された描画範囲
plt.xlim(x_min - k * x_std, x_max + k * x_std)
plt.ylim(y_min - k * y_std, y_max + k * y_std)

plt.show()

In [ ]:
# generated_data_original: shape (1000, 50, 2)
# 生成されたデータのデータセットごとの相関係数を計算
covs_of_generated_data = np.array([np.cov(sample.T) for sample in generated_data_original])  # shape: (1000, 2, 2)
# 相関係数を計算
correlation_coeffs = np.array([np.corrcoef(sample.T) for sample in generated_data_original])  # shape: (1000, 2, 2)
# print("生成されたデータのデータセットごとの相関係数(covs_of_generated_data)", covs_of_generated_data)
print("生成されたデータのデータセットごとの相関係数(covs_of_generated_data)のshape", covs_of_generated_data.shape)

# # 相関係数の分布
# plt.figure(figsize=(6, 4))
# plt.hist(covs_of_generated_data, bins=30, color='orchid')
# plt.title('Diffusion Model Sampling Correlation Coefficients')
# plt.xlabel('Correlation Coefficient')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()

In [ ]:
###################################################################


from scipy.stats import gaussian_kde
from mpl_toolkits.mplot3d import Axes3D  # 必要


# サンプリングの可視化
for seed, generated_samples in zip(random_seed, generated_data_list):
    print("############################################")
    print("Random Seed:", seed) # 開始の合図
    print("############################################")

    # ---------------------統計量制御---------------------
    generated_samples_mean_vecs = np.mean(generated_samples, axis=1) # 拡散モデルによるサンプルの平均ベクトル（1000, 2）
    generated_samples_cov_mats = np.array([np.cov(sample.T) for sample in generated_samples]) # 拡散モデルによるサンプルの共分散行列（1000, 2, 2）
    generated_samples_corr_coefs = np.array([np.corrcoef(sample.T)[0, 1] for sample in generated_samples]) # 拡散モデルによるサンプルの相関係数（1000,）

    # サイズの確認
    # print("Bootstrap Mean Vector", bootstrap_mean_vecs.shape) # 平均ベクトル
    # print("Bootstrap Covariance Matrix", bootstrap_cov_mats.shape) # 共分散行列
    # print("Bootstrap Correlation Coefficient", bootstrap_corr_coefs.shape) # 相関係数

    # 代表値を出力
    print("平均ベクトルの平均:", np.mean(generated_samples_mean_vecs, axis=0))
    print("共分散行列の平均:\n", np.mean(generated_samples_cov_mats, axis=0))
    print("相関係数の平均:", np.mean(generated_samples_corr_coefs))





    # 平均ベクトルの分布
    
    plt.figure(figsize=(6, 6))
    plt.scatter(generated_samples_mean_vecs[:, 0], generated_samples_mean_vecs[:, 1], alpha=0.5, color='teal', label='Diffusion Model Sampling Means')

    # 原点を基準として赤十字
    plt.axhline(original_mean[0], color='gray', linestyle='--')
    plt.axvline(original_mean[1], color='gray', linestyle='--')
    plt.scatter(original_mean[0], original_mean[1], color='red', marker='x', s=100, label='True Mean')

    plt.xlabel('Mean of X1')
    plt.ylabel('Mean of X2')
    plt.title('Scatter Plot of Diffusion Model Sampling Mean Vectors')
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()


    ## 可視化
    x_vals = means_of_generated_data[:, 0].flatten()
    y_vals = means_of_generated_data[:, 1].flatten()

    # 最小・最大 + 標準偏差から範囲を決定
    x_min, x_max = x_vals.min(), x_vals.max()
    y_min, y_max = y_vals.min(), y_vals.max()
    x_std, y_std = x_vals.std(), y_vals.std()

    k = 0.5  # 余白のスケール（標準偏差単位）

    # プロット
    plt.figure(figsize=(6, 6))
    plt.scatter(x_vals, y_vals, alpha=0.5, label='Generated Data')
    plt.title('Generated Data (Seed: {})'.format(seed))
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.legend()
    plt.grid()
    plt.axis('equal')

    # 拡張された描画範囲
    plt.xlim(x_min - k * x_std, x_max + k * x_std)
    plt.ylim(y_min - k * y_std, y_max + k * y_std)

    plt.show()


    # カーネル密度推定
    kde = gaussian_kde(generated_samples_mean_vecs.T)

    # グリッド生成
    x = np.linspace(np.min(generated_samples_mean_vecs[:, 0]) - 0.5, np.max(generated_samples_mean_vecs[:, 0]) + 0.5, 100)
    y = np.linspace(np.min(generated_samples_mean_vecs[:, 1]) - 0.5, np.max(generated_samples_mean_vecs[:, 1]) + 0.5, 100)
    X, Y = np.meshgrid(x, y)
    positions = np.vstack([X.ravel(), Y.ravel()])
    Z = kde(positions).reshape(X.shape)

    # 3Dプロット
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.9)

    ax.set_xlabel('Mean of X1')
    ax.set_ylabel('Mean of X2')
    ax.set_zlabel('Density')
    ax.set_title('Bird\'s Eye View of Diffusion Model Sampling Mean Vectors (KDE)')
    plt.tight_layout()
    plt.show()




    # 相関係数の分布
    plt.figure(figsize=(6, 4))
    plt.hist(generated_samples_corr_coefs, bins=30, color='orchid')
    plt.title('Diffusion Model Sampling Correlation Coefficients')
    plt.xlabel('Correlation Coefficient')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()




    # 終了の合図
    print("############################################")
    print("End")
    print("############################################")

    # 改行
    print("\n")

# GMMを試す

## いろんなGMM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_gmm_2d_toy_dataset(num_samples_per_mode=500,
                                mu1=np.array([-3.0, -3.0]),
                                sigma1=np.array([[1.0, 0.5], [0.5, 1.0]]),
                                mu2=np.array([3.0, 3.0]),
                                sigma2=np.array([[1.0, -0.5], [-0.5, 1.0]]),
                                random_seed=42):
    """
    2つの2次元正規分布を組み合わせたガウス混合モデル (GMM) からトイデータセットを生成します。

    Args:
        num_samples_per_mode (int): 各正規分布から生成するサンプル数。
        mu1 (np.ndarray): 1つ目の正規分布の平均ベクトル (2次元)。
        sigma1 (np.ndarray): 1つ目の正規分布の共分散行列 (2x2)。
        mu2 (np.ndarray): 2つ目の正規分布の平均ベクトル (2次元)。
        sigma2 (np.ndarray): 2つ目の正規分布の共分散行列 (2x2)。
        random_seed (int): 乱数生成のシード。

    Returns:
        np.ndarray: 生成されたデータポイント (N x 2)。
    """
    np.random.seed(random_seed)

    # 1つ目の正規分布からデータを生成
    data_mode1 = np.random.multivariate_normal(mu1, sigma1, num_samples_per_mode)

    # 2つ目の正規分布からデータを生成
    data_mode2 = np.random.multivariate_normal(mu2, sigma2, num_samples_per_mode)

    # 両方のデータを結合
    dataset = np.vstack((data_mode1, data_mode2))

    return dataset

def plot_2d_dataset(dataset, title="2D GMM Toy Dataset"):
    """
    2次元データセットをプロットします。

    Args:
        dataset (np.ndarray): プロットするデータセット (N x 2)。
        title (str): グラフのタイトル。
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(dataset[:, 0], dataset[:, 1], s=10, alpha=0.7)
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.axis('equal') # x軸とy軸のスケールを合わせる
    plt.show()

if __name__ == "__main__":
    # データセットの生成
    # 例1: 離れた2つのクラスタ
    print("Generating Dataset 1: Two distinct clusters...")
    dataset1 = generate_gmm_2d_toy_dataset(
        num_samples_per_mode=1000,
        mu1=np.array([-5.0, -5.0]),
        sigma1=np.array([[1.5, 0.2], [0.2, 1.5]]),
        mu2=np.array([5.0, 5.0]),
        sigma2=np.array([[1.5, -0.2], [-0.2, 1.5]])
    )
    plot_2d_dataset(dataset1, title="2D GMM Toy Dataset (Distinct Clusters)")

    # 例2: 少し重なり合う2つのクラスタ
    print("\nGenerating Dataset 2: Two slightly overlapping clusters...")
    dataset2 = generate_gmm_2d_toy_dataset(
        num_samples_per_mode=750,
        mu1=np.array([-2.0, 0.0]),
        sigma1=np.array([[1.0, 0.8], [0.8, 1.0]]),
        mu2=np.array([2.0, 0.0]),
        sigma2=np.array([[1.0, -0.8], [-0.8, 1.0]])
    )
    plot_2d_dataset(dataset2, title="2D GMM Toy Dataset (Slightly Overlapping Clusters)")

    # 例3: 異なるサイズのクラスタ (サンプル数を変える)
    print("\nGenerating Dataset 3: Clusters with different number of samples...")
    # num_samples_per_modeを引数で渡すが、実際には異なる数で生成する場合は関数を少し変更するか、
    # 各modeのデータ生成を別々に呼び出す必要がある。
    # ここでは便宜上、同じ数で生成。
    dataset3_mode1 = np.random.multivariate_normal(np.array([0.0, 4.0]), np.array([[0.8, 0.1], [0.1, 0.8]]), 300)
    dataset3_mode2 = np.random.multivariate_normal(np.array([0.0, -4.0]), np.array([[2.0, 0.0], [0.0, 0.5]]), 1500)
    dataset3 = np.vstack((dataset3_mode1, dataset3_mode2))
    plot_2d_dataset(dataset3, title="2D GMM Toy Dataset (Different Sample Counts)")

    # 生成されたデータセットの形状を確認
    print(f"\nShape of Dataset 1: {dataset1.shape}")
    print(f"Shape of Dataset 2: {dataset2.shape}")
    print(f"Shape of Dataset 3: {dataset3.shape}")

    # このデータセットを拡散モデルの学習ループに渡すことを想定
    # 例: dataloader = create_dataloader(dataset)

## 無相関な正規分布からできたGMM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_uncorrelated_gmm_2d_toy_dataset(num_total_samples=2000,
                                            ratio_mode1=0.5, # モード1のデータが全体の何割を占めるか (0.0 ~ 1.0)
                                            mu1=np.array([-3.0, -3.0]),
                                            sigma1_diag=np.array([1.0, 1.0]), # 対角成分のみ指定
                                            mu2=np.array([3.0, 3.0]),
                                            sigma2_diag=np.array([1.0, 1.0]), # 対角成分のみ指定
                                            random_seed=42):
    """
    2つの無相関な2次元正規分布を組み合わせたガウス混合モデル (GMM) からトイデータセットを生成します。
    各正規分布の比率を指定できます。

    Args:
        num_total_samples (int): 生成する総サンプル数。
        ratio_mode1 (float): モード1 (1つ目の正規分布) のデータが全体の何割を占めるか (0.0 ~ 1.0)。
                              モード2の比率は (1 - ratio_mode1) になります。
        mu1 (np.ndarray): 1つ目の正規分布の平均ベクトル (2次元)。
        sigma1_diag (np.ndarray): 1つ目の正規分布の共分散行列の対角成分 (2要素)。
                                  非対角成分は0と仮定されます。
        mu2 (np.ndarray): 2つ目の正規分布の平均ベクトル (2次元)。
        sigma2_diag (np.ndarray): 2つ目の正規分布の共分散行列の対角成分 (2要素)。
                                  非対角成分は0と仮定されます。
        random_seed (int): 乱数生成のシード。

    Returns:
        np.ndarray: 生成されたデータポイント (N x 2)。
    """
    if not (0.0 <= ratio_mode1 <= 1.0):
        raise ValueError("ratio_mode1 must be between 0.0 and 1.0")

    np.random.seed(random_seed)

    # 各モードのサンプル数を計算
    num_samples_mode1 = int(num_total_samples * ratio_mode1)
    num_samples_mode2 = num_total_samples - num_samples_mode1 # 残りはモード2

    # 共分散行列を作成 (対角成分のみ有効)
    sigma1 = np.diag(sigma1_diag)
    sigma2 = np.diag(sigma2_diag)

    print(f"Generating {num_total_samples} samples:")
    print(f"  Mode 1 ({ratio_mode1*100:.1f}%): {num_samples_mode1} samples")
    print(f"  Mode 2 ({(1-ratio_mode1)*100:.1f}%): {num_samples_mode2} samples")

    # 1つ目の正規分布からデータを生成
    data_mode1 = np.random.multivariate_normal(mu1, sigma1, num_samples_mode1)

    # 2つ目の正規分布からデータを生成
    data_mode2 = np.random.multivariate_normal(mu2, sigma2, num_samples_mode2)

    # 両方のデータを結合
    dataset = np.vstack((data_mode1, data_mode2))

    # データをシャッフル (モードの偏りをなくすため)
    np.random.shuffle(dataset)

    return dataset

def plot_2d_dataset(dataset, title="2D GMM Toy Dataset (Uncorrelated)"):
    """
    2次元データセットをプロットします。

    Args:
        dataset (np.ndarray): プロットするデータセット (N x 2)。
        title (str): グラフのタイトル。
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(dataset[:, 0], dataset[:, 1], s=10, alpha=0.7)
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.axis('equal') # x軸とy軸のスケールを合わせる
    plt.show()

if __name__ == "__main__":
    # 例1: 比率50:50、円形クラスタ
    print("--- Example 1: 50:50 Ratio, Circular Clusters ---")
    dataset1 = generate_uncorrelated_gmm_2d_toy_dataset(
        num_total_samples=2000,
        ratio_mode1=0.5,
        mu1=np.array([-4.0, -4.0]),
        sigma1_diag=np.array([1.0, 1.0]),
        mu2=np.array([4.0, 4.0]),
        sigma2_diag=np.array([1.0, 1.0])
    )
    plot_2d_dataset(dataset1, title="Uncorrelated GMM (50:50, Circular)")

    # 例2: 比率80:20、楕円形クラスタ (軸に平行)
    print("\n--- Example 2: 80:20 Ratio, Elliptical Clusters ---")
    dataset2 = generate_uncorrelated_gmm_2d_toy_dataset(
        num_total_samples=3000,
        ratio_mode1=0.8,
        mu1=np.array([-2.0, 0.0]),
        sigma1_diag=np.array([0.5, 2.0]), # X軸方向に狭く、Y軸方向に広い楕円
        mu2=np.array([2.0, 0.0]),
        sigma2_diag=np.array([2.0, 0.5])  # X軸方向に広く、Y軸方向に狭い楕円
    )
    plot_2d_dataset(dataset2, title="Uncorrelated GMM (80:20, Elliptical)")

    # 例3: 比率20:80、異なる広がり
    print("\n--- Example 3: 20:80 Ratio, Different Spreads ---")
    dataset3 = generate_uncorrelated_gmm_2d_toy_dataset(
        num_total_samples=2500,
        ratio_mode1=0.2,
        mu1=np.array([0.0, 5.0]),
        sigma1_diag=np.array([0.8, 0.8]), # 小さめの円
        mu2=np.array([0.0, -5.0]),
        sigma2_diag=np.array([3.0, 3.0])  # 大きめの円
    )
    plot_2d_dataset(dataset3, title="Uncorrelated GMM (20:80, Different Spreads)")

    # 生成されたデータセットの形状を確認
    print(f"\nShape of Dataset 1: {dataset1.shape}")
    print(f"Shape of Dataset 2: {dataset2.shape}")
    print(f"Shape of Dataset 3: {dataset3.shape}")

## 試したいGMM: 比率30:70、円形クラスタ

In [ ]:
# 例4: 比率30:70、円形クラスタ
print("--- Example 4: 30:70 Ratio, Circular Clusters ---")
dataset4 = generate_uncorrelated_gmm_2d_toy_dataset(
    num_total_samples=2000,
    ratio_mode1=0.3,
    mu1=np.array([-3.0, -3.0]),
    sigma1_diag=np.array([1.0, 1.0]),
    mu2=np.array([3.0, 3.0]),
    sigma2_diag=np.array([1.0, 1.0])
)
plot_2d_dataset(dataset4, title="Uncorrelated GMM (30:70, Circular)")

In [ ]:
# モデルの定義と拡散プロセスの実装
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import time
from sklearn.preprocessing import StandardScaler
# Diffusion Modelの定義
time_embed_dim = 16 # 時間埋め込みの次元数
model_4 = DiffusionModel(time_embed_dim=time_embed_dim).to(device)
optimizer_4 = Adam(model_4.parameters(), lr=lr)
# Diffuserのインスタンスを作成
diffuser_4 = Diffuser(num_timesteps=num_timesteps, device=device)

In [ ]:
# モデルの初期化
time_embed_dim = 16
model = DiffusionModel(time_embed_dim=time_embed_dim).to(device)
optimizer = Adam(model.parameters(), lr=lr)
diffuser = Diffuser(num_timesteps=num_timesteps, device=device)

# 学習データ(ガウスノイズ)
print("############################################")
print(f"Data_Set_{i+1}, Seed: {seed}") # 開始の合図

print(f"拡散ステップ数: {num_timesteps}, 学習エポック数: {epochs}, 学習率: {lr}")
print("############################################")
np.random.seed(seed) # 取得した乱数を新しいシード値として設定
data = np.random.multivariate_normal(original_mean, original_cov, size=dataset_size) # 学習元データの生成 (50, 2)
# データの標準化
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data) # shape: (500, 2), dtype: float64

print("data.shape", data.shape) # (50, 2)
# PyTorchテンソルへ明示的に float32 で変換
train_data = torch.tensor(scaled_data, dtype=torch.float32).to(device) # shape: (50, 2)

# データローダー作成
batch_size = 10
# モデル学習に使う DataLoader も float32 のテンソルから作成
dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)


# 原点を基準として赤十字
plt.axhline(original_mean[0], color='gray', linestyle='--')
plt.axvline(original_mean[1], color='gray', linestyle='--')
plt.scatter(original_mean[0], original_mean[1], color='red', marker='x', s=100, label='True Mean')
# データの可視化
plt.scatter(data[:, 0], data[:, 1], label='Original Data')
plt.title(f'Original Data (Seed: {seed})')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()
plt.xlim(0, 10)
plt.ylim(0, 10)
plt.grid()
plt.show()

# 標準化したデータの可視化
plt.scatter(scaled_data[:, 0], scaled_data[:, 1], label='Scaled Data')
plt.title(f'Scaled Data (Seed: {seed})')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.grid()
plt.show()

# データの要約
print("dataの平均ベクトル", np.mean(data, axis=0)) # 平均ベクトル
print("dataの分散共分散行列", np.cov(data.T)) # 分散共分散行列
print("dataの相関係数", np.corrcoef(data.T)) # 相関係数

# 学習
losses = []
for epoch in range(epochs):
    loss_sum = 0.0
    for batch in dataloader:
        optimizer.zero_grad()
        x = batch.to(device)
        t = torch.randint(1, num_timesteps + 1, (len(x),), device=device)

        x_noisy, noise = diffuser.add_noise(x, t)
        noise_pred = model(x_noisy, t)
        loss = F.mse_loss(noise_pred, noise)

        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
    avg_loss = loss_sum / len(dataloader)
    losses.append(avg_loss)
    # 5の倍数エポックで損失を表示
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}, Loss: {avg_loss}")
# 辞書に保存
models[f"model_{i+1}"] = model
original_datas[f"seed_{seed}"] = data
print("学習終了")
end_time = time.time() # 計測終了
print('\n')
print(f"学習時間: {end_time - start_time:.2f}秒")

# # モデルの保存
# torch.save(model.state_dict(), f"model_{i+1}.pth")

# 学習曲線のプロット
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Trained By data_by_seed_{}'.format(seed))
plt.show()
print('\n')
print("#"*50)
print('\n')